# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [1]:
import glob
import numpy as np
import pandas as pd
import scipy as sp
import scipy.io
import scipy.signal
import matplotlib.pyplot as plt
%matplotlib inline


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls


def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]


def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability.

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and
        corresponding reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse
        rate error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))


def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an
    aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return (AggregateErrorMetric(errs, confs), errs, confs)

fs = 125
window_length_s = 8
window_shift_s = 2
window_overlap_s = 6
window_length = window_length_s * fs
window_shift = window_shift_s * fs
window_overlap = window_overlap_s *fs
low_BPM = 40
low_Hz = low_BPM/60 
high_BPM = 240
high_Hz = high_BPM/60 

def LoadTroikaReferanceFile(ref_fl):
    """
    Loads and extracts signals from a troika referance data file.

    Args:
        ref_fls: (str) filepath to a troika .mat file.

    Returns:
        numpy array for reference values.
    """

    ref_data = sp.io.loadmat(ref_fl)['BPM0']
    return ref_data

def BandpassFilter(signal, fs=125, low_Hz=0.67, high_Hz=4):
    """
    Bandpass filter the signal between 40 & 240 BPM.
    
    Args:
        signal: Numpy array to filter
        fs: sample rate
        pass band:-
        low_Hz: low frequency bound
        high_Hz: high frequency bound
        
    Returns:
        Numpy array to filtered signal
        """
    b, a = sp.signal.butter(3, (low_Hz, high_Hz), btype='bandpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)

def Featurize(signal, fs, window_shift):
    """featurization of the accelerometer signal.
    
    Args:
        signal: input signal             
        fs: sampling rate       
        
    Returns:
        n-tuple of accelerometer features
    """
    n = len(signal)
    window_length = window_length_s * fs
    B = (n - window_length) // window_shift + 1
    spec = []
    for i in range(B):
        wind = signal[i * window_shift:i*window_shift+window_length]
        fft = np.abs(np.fft.rfft(wind))
        spec.append(fft)
    spec = np.stack(spec, axis=1)
    freq = np.fft.rfftfreq(window_length, 1/fs)
    return spec, freq

def Estimates(ppg_spec, ppg_freqs, est):
    #Calculate the estimation of the heart rate
    j = 7
    i = len(est)
    if i != 0:
        est_range = np.argsort(ppg_spec[:, i], axis=0)[-7:]
        estimation = ppg_freqs[est_range[np.argmin(np.abs(ppg_freqs[est_range] - (est[-1]/60)))]] * 60
    else:
        estimation = ppg_freqs[np.argmax(ppg_spec[:,0], axis=0)] * 60
    return estimation

def RunPulseRateAlgorithm(data_fl, ref_fl):
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    ref_data = LoadTroikaReferanceFile(ref_fl)
    # Bandpass all the signals
    ppg = BandpassFilter(ppg)
    accx = BandpassFilter(accx)
    accy = BandpassFilter(accy)
    accz = BandpassFilter(accz)
    
    acc = np.sqrt(accx**2 + accy**2 + accz**2)
    errors = []
    confidence = []
    est = []

    acc_spec, acc_freq = Featurize(acc, fs, window_shift)
    ppg_spec, ppg_freqs = Featurize(ppg, fs, window_shift)

    
    for i in range(len(ppg_spec[0,])):
        est = np.append(est, Estimates(ppg_spec, ppg_freqs, est))

    for i, estimation in enumerate(est):
        confidence = np.append(confidence, CalcConf(estimation, ppg_spec[:, i], ppg_freqs))
        
    errors = np.abs(np.diag(np.array(est) - ref_data))
    return errors, np.array(confidence)

def CalcConf(pr_bpm, fft, freqs):
    #Calculate the confidence of the heart rate
    pr_freq = pr_bpm/60 #from Hz to BPM
    pulse_rate = (freqs>=(pr_freq-0)) & (freqs<=(pr_freq+0))
    signal_power = np.sum(np.abs(fft[pulse_rate]))/np.sum(np.abs(fft))

    return signal_power